In [21]:
import re
import pandas as pd

df = pd.read_csv("pronoun_testcases.csv")

In [22]:
COMMON_NOUNS = {
    'book', 'laptop', 'car', 'attitude', 'trip', 'bag', 'pen',
    'phone', 'room', 'shoes', 'idea', 'warning'
}

In [23]:
def smart_pronoun_transform(text, target_gender):
    words = re.findall(r"\w+|[^\w\s]", text)
    transformed = []

    for i, word in enumerate(words):
        lower = word.lower()

        if target_gender == 'female':
            if lower == 'he':
                new_word = 'she'
            elif lower == 'him':
                new_word = 'her'
            elif lower == 'his':
                if i + 1 < len(words) and re.match(r'\w+', words[i + 1]):
                    new_word = 'her'
                else:
                    new_word = 'his'
            elif lower == 'himself':
                new_word = 'herself'
            else:
                new_word = word

        elif target_gender == 'male':
            if lower == 'she':
                new_word = 'he'
            elif lower == 'her':
                next_word = words[i + 1].lower() if i + 1 < len(words) and re.match(r'\w+', words[i + 1]) else ""
                if next_word in COMMON_NOUNS:
                    new_word = 'his'
                else:
                    new_word = 'him'
            elif lower == 'hers':
                new_word = 'his'
            elif lower == 'herself':
                new_word = 'himself'
            else:
                new_word = word
        else:
            new_word = word

        if word[0].isupper():
            new_word = new_word.capitalize()

        transformed.append(new_word)

    return ''.join([
        t if re.match(r"\W", t) or i == 0 or re.match(r"\W", transformed[i - 1]) else ' ' + t
        for i, t in enumerate(transformed)
    ])

In [24]:
df['predicted_output'] = df.apply(
    lambda row: smart_pronoun_transform(row['input_text'], row['target_gender']),
    axis=1
)

In [25]:
df['correct'] = df['predicted_output'] == df['expected_output']
correct = df['correct'].sum()
total = len(df)
accuracy = correct / total * 100

print(f"\n Accuracy: {correct}/{total} correct ({accuracy:.2f}%)")


 Accuracy: 26/26 correct (100.00%)
